This Notebook is intended for grading purposes

First, we will install all the modules and packages.

It is important to note that these tensorflow packages only work on Python versions < 3.7

In [1]:
import cv2
import os
import pandas as pd
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.models import load_model
from tensorflow.keras.applications.mobilenet_v2 import preprocess_input
import numpy as np

%pylab inline 
from IPython.display import clear_output
import time
import PIL.Image
from io import BytesIO
import IPython.display
from mtcnn.mtcnn import MTCNN
import keras

Using TensorFlow backend.


Populating the interactive namespace from numpy and matplotlib


Next we make sure all the directories we might need are ready to be called on:

In [5]:
NOTEBOOK_DIR = !pwd
NOTEBOOK_DIR = NOTEBOOK_DIR[0]
ROOT_DIR = os.path.abspath(os.path.join(NOTEBOOK_DIR, os.pardir))
DATA_DIR = os.path.join(ROOT_DIR, 'data')
IMAGES_DIR = os.path.join(DATA_DIR, 'images')
ANNOTATIONS_DIR = os.path.join(DATA_DIR, 'annotations')
SUB_IMAGES_DIR = os.path.join(IMAGES_DIR, 'sub')
MODEL_DIR = os.path.join(ROOT_DIR, 'models')
MNM_MODEL_DIR = os.path.join(MODEL_DIR, 'keras_mask_or_no_mask_model')
CINC_WORN_MASKS_DIR = os.path.join(MODEL_DIR, 'keras_8_feature_incorrect_mask_sequential_model')


In [6]:
train_df = pd.read_csv(os.path.join(DATA_DIR, 'img_data/kaggle_training.csv'), index_col=False)
train_df.rename(
    columns={'x1': 'x', 'x2': 'y', 'y1': 'w', 'y2': 'h'},
    inplace=True
)
train_df.sort_values('name', axis = 0, inplace = True)
features = [
    'face_with_mask', 
    'face_no_mask', 
]
train_df = train_df[train_df['classname'].isin(features)]
train_df['classname'].unique()

array(['face_no_mask', 'face_with_mask'], dtype=object)

In [7]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

model = keras.models.load_model(MNM_MODEL_DIR)
labeler = LabelEncoder()
detector = MTCNN()

y = list(set(train_df['classname']))

y = labeler.fit_transform(y)

y = to_categorical(y)
y

array([[1., 0.],
       [0., 1.]], dtype=float32)

So you can see our failures in LIVE-TIME :D

In [ ]:
CYAN = (0, 255, 255)
GREEN = (0, 255, 0)
MAGENTA = (255, 0, 255)

cap = cv2.VideoCapture(0)
rval, frame = cap.read()

session_data = list()
count = 1
while cap.isOpened(): 
    #Capture frame-by-frame
#     ret, frame = cap.read()
#     frame = get_frame(cap)

    if frame is not None:
        grayFrame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
#         image = array_to_image(frame)

        #Use MTCNN to detect faces
        faces = detector.detect_faces(grayFrame)
        if faces != []:
            
            data = list()
            results = list()
            frame_df = pd.DataFrame()
            for face in faces:
#                 data.append([count, face['box']])
                bounding_box = face['box']
                keypoints = face['keypoints']
                x, y, w, h = bounding_box
                img = frame[y:y+h,x:x+w]

                try:
                    new_img = cv2.resize(img, (150, 150))
                    new_img = new_img.reshape(-1, 50, 50, 1)
                except Exception as e:
                    print(str(e))
                prediction = model.predict(new_img)
#                 print(prediction)
#                 print("PREDICTION: {}".format(labeler.inverse_transform(prediction)))
                data.append([count, bounding_box, prediction])
                
                cv2.circle(frame,(keypoints['left_eye']), 2, CYAN, 2)
                cv2.circle(frame,(keypoints['right_eye']), 2, CYAN, 2)
                cv2.circle(frame,(keypoints['nose']), 2, CYAN, 2)
                cv2.circle(frame,(keypoints['mouth_left']), 2, CYAN, 2)
                cv2.circle(frame,(keypoints['mouth_right']), 2, CYAN, 2)
                
            print(data)
            results += [i for i in data if len(i) == 1]
            results += [[j for j in i] for i in data if len(i) > 1]

#             print(results)
            image = []
            classname = []
            for k, i, j in results:
                cn = np.argmax(j)
                classname.append(cn if cn < 2 else 0)
                image.append(i)
            
            
            print("FRAME NO.: {}\nCLASSNAME ARGMAX: {}\nINVERSE TRANSFORM: {}\n".format(
                count, classname, labeler.inverse_transform(classname)
            ))
            frame_df['image'] = image
            frame_df['classname'] = classname
            frame_df['classname'] = labeler.inverse_transform(frame_df['classname'])
            print(frame_df)


            for i in range(len(frame_df)):
                x, y, w, h = frame_df.iloc[i]['image']
                cname = str(frame_df.iloc[i]['classname'])
               
                if cname == 'face_with_mask':
                    label = "Mask"
                else:
                    label = "No Mask"
                color = GREEN if label == 'Mask' else MAGENTA
                cv2.putText(frame, label, (x, y- 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.45, color, 4)
                        
                cv2.rectangle(frame, (x, y), (x + w, y + h),color, 2)

            session_data.append(results)


        #display resulting frame
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        cv2.imshow('frame', frame)
#         plt.imshow(frame, interpolation = 'bicubic')
#         print(frame.shape)
        

    ret, frame = cap.read()
    clear_output(wait=True)
    count += 1
    if cv2.waitKey(1) &0xFF == ord('q'):
        break
#When everything's done, release capture
cap.release()
cv2.destroyAllWindows()

[[70, [577, 139, 260, 323], array([[0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [2.0197724e-13, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [0.0000000e+00, 1.0000000e+00],
       [1.0000000e+00, 0.0000000e+00],
       [1.0000000e+00, 0.0000000e+00